In [1]:
# Loading relevant data packages
import pandas as pd
import requests
import json
import csv
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
%matplotlib inline
pd.set_option('display.max_columns', None)



In [29]:
# API parameters
api_key = "04add5b7cdmshd4b470181d45798p1428bbjsn3f4be7e144f4"
api_host = "tripadvisor16.p.rapidapi.com"
url = "https://tripadvisor16.p.rapidapi.com/api/v1/flights/searchFlights"

# API customer inputs (this will come from UI when ready)

sourceAirportCode = 'EWR'
destinationAirportCode = 'LAS'
outDate= "2024-06-22"
itineraryType= "ONE_WAY"
sortOrder= 'PRICE'
classOfService= "ECONOMY"
pageNumber= '1'
currencyCode= 'USD'

#sourceAirportCode = "BOI"
#destinationAirportCode = "TPA"
#outDate = "2024-06-22"
#itineraryType = "ROUND_TRIP"
#sortOrder = "PRICE"
#returnDate = "2024-06-29"
#numAdults = "1"
#numSeniors = "0"
#classOfService = "ECONOMY"
#pageNumber = "1" # for now fetch just first page to save on query number

# build API inputs
querystring = {"sourceAirportCode":sourceAirportCode,"destinationAirportCode":destinationAirportCode,\
               "date":outDate, "itineraryType":itineraryType,"sortOrder":sortOrder,\
               "classOfService":classOfService,"pageNumber":pageNumber, "currencyCode":currencyCode}

headers = {"X-RapidAPI-Key": api_key,\
           "X-RapidAPI-Host": api_host}

In [30]:
# fetch data from API
response = requests.get(url, headers=headers, params=querystring)

In [31]:
# convert json to python dict
response_text = json.loads(response.text)

In [32]:
# # cell to visualize data structure for debug purpose
# response_text['data']['flights'][0]['segments'][1]['legs'][0]
# response_text['data']['flights'][2]['purchaseLinks'][0]
#response_text

In [33]:
# Extract flights information
flights_data = []

for flight in response_text['data']['flights']:
    segments = flight['segments']
    for segment in segments:
        legs = segment['legs']
        for leg in legs:
            departure_time = datetime.fromisoformat(leg['departureDateTime'])
            arrival_time = datetime.fromisoformat(leg['arrivalDateTime'])
            air_time_minutes = (arrival_time - departure_time).total_seconds() / 60
            
            departure_datetime = datetime.fromisoformat(leg['departureDateTime'])
            fl_date = departure_datetime.strftime("%Y-%m-%d")
            
            flight_info = {
                'FL_DATE': str(fl_date),
                'AIRLINE': str(flight['purchaseLinks'][0]['partnerSuppliedProvider']['displayName']), 
                'AIRLINE_CODE': leg['operatingCarrier']['code'],
                'DOT_CODE': 0,
                'FL_NUMBER': leg['flightNumber'],
                'ORIGIN': leg['originStationCode'],
                'DEST': leg['destinationStationCode'],
                'AIR_TIME': air_time_minutes,
                'DISTANCE': leg['distanceInKM'],
                #'total_price': flight['purchaseLinks'][0]['totalPrice'],  # Assuming only one purchase link per flight
                'ARR_DELAY': "0",
                'CANCELLED': 0
                
            }
            flights_data.append(flight_info)

# Create DataFrame
flights_df = pd.DataFrame(flights_data)
flights_df

,FL_DATE,AIRLINE,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,DEST,AIR_TIME,DISTANCE,ARR_DELAY,CANCELLED
0,2024-06-22,eDreams,NK,0,465,EWR,LAS,326.0,3579.11990,0,0
1,2024-06-22,eDreams,NK,0,440,EWR,LAS,327.0,3579.11990,0,0
2,2024-06-22,eDreams,NK,0,2322,EWR,LAX,363.0,3944.68680,0,0
3,2024-06-22,eDreams,NK,0,507,LAX,LAS,82.0,380.66320,0,0
4,2024-06-22,FareScan,NK,0,2322,EWR,LAX,363.0,3944.68680,0,0
5,2024-06-22,FareScan,NK,0,2699,LAX,LAS,83.0,380.66320,0,0
6,2024-06-22,eDreams,NK,0,993,EWR,FLL,186.0,1718.29930,0,0
7,2024-06-22,eDreams,NK,0,777,FLL,LAS,311.0,3495.97680,0,0
8,2024-06-22,eDreams,NK,0,521,EWR,FLL,185.0,1718.29930,0,0
9,2024-06-22,eDreams,NK,0,777,FLL,LAS,311.0,3495.97680,0,0


### THE COLUMNS BELOW ARE NEEDED FOR THE INPUT ###

In [34]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   FL_DATE       19 non-null     object 
 1   AIRLINE       19 non-null     object 
 2   AIRLINE_CODE  19 non-null     object 
 3   DOT_CODE      19 non-null     int64  
 4   FL_NUMBER     19 non-null     int64  
 5   ORIGIN        19 non-null     object 
 6   DEST          19 non-null     object 
 7   AIR_TIME      19 non-null     float64
 8   DISTANCE      19 non-null     float64
 9   ARR_DELAY     19 non-null     object 
 10  CANCELLED     19 non-null     int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 1.8+ KB


In [35]:
ml_api_key = 'JAGEV14Z3dQnAwKvWfOISA6pt7eyD5We'

In [36]:
import pandas as pd
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True)

# Your dataframe
flights_df


# Convert dataframe to a list of dictionaries
data_to_send = flights_df.to_dict(orient='records')

data = {
    "Inputs": {
        "input1": data_to_send
    },
    "GlobalParameters": {}
}

body = str.encode(json.dumps(data))

url = 'http://fd1fb9e2-af8c-4f79-b6f6-6ceeae57dbf8.eastus2.azurecontainer.io/score'
api_key = ml_api_key # You need to define ml_api_key

if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print(result)
    # Decode bytes to string
    json_string = result.decode('utf-8')
    # Convert string to dictionary
    data = json.loads(json_string)
    # Extract the list of dictionaries from the JSON
    results = data['Results']['WebServiceOutput0']
    # Convert the list of dictionaries to DataFrame
    result_df = pd.DataFrame(results)
    result_df
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'{"Results": {"WebServiceOutput0": [{"FL_DATE": "2024-06-22", "AIRLINE": "eDreams", "AIRLINE_CODE": "NK", "DOT_CODE": 0, "FL_NUMBER": 465, "ORIGIN": "EWR", "DEST": "LAS", "AIR_TIME": 326.0, "DISTANCE": 3579.1199, "ARR_DELAY": 0.0, "CANCELLED": 0, "Scored Labels": 1, "Scored Probabilities": 0.6592648830762823}, {"FL_DATE": "2024-06-22", "AIRLINE": "eDreams", "AIRLINE_CODE": "NK", "DOT_CODE": 0, "FL_NUMBER": 440, "ORIGIN": "EWR", "DEST": "LAS", "AIR_TIME": 327.0, "DISTANCE": 3579.1199, "ARR_DELAY": 0.0, "CANCELLED": 0, "Scored Labels": 1, "Scored Probabilities": 0.6592648830762823}, {"FL_DATE": "2024-06-22", "AIRLINE": "eDreams", "AIRLINE_CODE": "NK", "DOT_CODE": 0, "FL_NUMBER": 2322, "ORIGIN": "EWR", "DEST": "LAX", "AIR_TIME": 363.0, "DISTANCE": 3944.6868, "ARR_DELAY": 0.0, "CANCELLED": 0, "Scored Labels": 1, "Scored Probabilities": 0.6185342240110705}, {"FL_DATE": "2024-06-22", "AIRLINE": "eDreams", "AIRLINE_CODE": "NK", "DOT_CODE": 0, "FL_NUMBER": 507, "ORIGIN": "LAX", "DEST": "LAS",

In [37]:
result_df

,FL_DATE,AIRLINE,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,DEST,AIR_TIME,DISTANCE,ARR_DELAY,CANCELLED,Scored Labels,Scored Probabilities
0,2024-06-22,eDreams,NK,0,465,EWR,LAS,326.0,3579.11990,0.0,0,1,0.659265
1,2024-06-22,eDreams,NK,0,440,EWR,LAS,327.0,3579.11990,0.0,0,1,0.659265
2,2024-06-22,eDreams,NK,0,2322,EWR,LAX,363.0,3944.68680,0.0,0,1,0.618534
3,2024-06-22,eDreams,NK,0,507,LAX,LAS,82.0,380.66320,0.0,0,1,0.685791
4,2024-06-22,FareScan,NK,0,2322,EWR,LAX,363.0,3944.68680,0.0,0,1,0.618534
5,2024-06-22,FareScan,NK,0,2699,LAX,LAS,83.0,380.66320,0.0,0,1,0.685791
6,2024-06-22,eDreams,NK,0,993,EWR,FLL,186.0,1718.29930,0.0,0,1,0.618534
7,2024-06-22,eDreams,NK,0,777,FLL,LAS,311.0,3495.97680,0.0,0,1,0.594884
8,2024-06-22,eDreams,NK,0,521,EWR,FLL,185.0,1718.29930,0.0,0,1,0.618534
9,2024-06-22,eDreams,NK,0,777,FLL,LAS,311.0,3495.97680,0.0,0,1,0.594884
